# **INAIL publications**

Uses Selenium with Google Chrome to maintain session continuity and bypass dynamic loading or access checks on INAIL’s catalog, ensuring stable navigation across search pages and article detail pages.


In [1]:
# Here we install Google Chrome
%pip install google-colab-selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.7/512.7 kB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 99.5 MB/s eta 0:00:00


In [2]:
!pip install PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 54.1 MB/s eta 0:00:00


In [27]:
import google_colab_selenium as gs
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import fitz  # PyMuPDF
import re
import os
import time

# We start Google Chrome in the headless mode (without user interface)
options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36')

driver = gs.Chrome(options=options)

<IPython.core.display.Javascript object>

**Generic scraping function**

Implements a function to extract metadata and full PDF text from an INAIL article page, handling optional fields gracefully. Adapts to missing descriptions or shifted field positions so titles, dates, PDF links, and other details are parsed reliably.


In [4]:
# Funzione per scaricare PDF dal link e estrapoliamo il testo

def scarica_e_leggi_pdf(pdf_url):
    nome_file_pdf = 'inail_temp.pdf'

    # Download con wget (ignora SSL, funziona sempre)
    os.system(f'wget -O {nome_file_pdf} "{pdf_url}"')

    if not os.path.exists(nome_file_pdf):
        print("Errore nel download del PDF.")
        return None

    with fitz.open(nome_file_pdf) as doc:
        full_text = " ".join(page.get_text() for page in doc)

    full_text = re.sub(r'\s+', ' ', full_text).strip()

    # Pulisci file temp (opzionale)
    os.remove(nome_file_pdf)

    return full_text

In [5]:
# Funzione per estrazione metadati e testo PDF da una pagina di articolo INAIL
# Attenzione però, non tutti i link contengo necessariamente una descrizione causando quindi un cambio posizione per le altre informazioni, pertanto serve una funzione che si adatti a questi casi specifii

def scrape_inail_details(driver, url):
    driver.get(url)
    time.sleep(3)
    page = BeautifulSoup(driver.page_source, "lxml")

    # Titolo
    title_elem = page.find("h2", class_="h1")
    title = title_elem.get_text(strip=True) if title_elem else "N/A"

    # Blocchi descrizione, in cui sono contenute piu informazioni utili
    descr_blocks = page.find_all("p", class_="text-20")

    # Abstract: primo sostanzioso (indice 1 fisso)
    abstract = ""
    if len(descr_blocks) > 1:
        abstract = descr_blocks[1].get_text(strip=True)
    elif descr_blocks:
        abstract = descr_blocks[0].get_text(strip=True)

    # Descrizione: secondo blocco, se sostanzioso e non metadati
    descrizione = "N/A"
    if len(descr_blocks) > 2:
        candidate = descr_blocks[2].get_text(strip=True)
        if len(candidate) > 50 and "Prodotto:" not in candidate and "Edizioni:" not in candidate:
            descrizione = candidate

    # Prodotto: cerca dinamicamente, con regex per estrarre solo il valore
    prodotto = "N/A"
    for block in descr_blocks:
        block_text = block.get_text(strip=True)
        if "Prodotto:" in block_text:
            # Regex: cattura dopo "Prodotto:" fino al prossimo ":" o fine
            match = re.search(r'Prodotto:\s*([^:]+?)(?=\s*(Edizioni|Disponibilità|Info):|$)', block_text, re.IGNORECASE)
            if match:
                prodotto = match.group(1).strip()
            break

    # Data pubblicazione
    data_elem = page.find("strong", class_="js-date-value")
    data_pub = ""
    if data_elem:
        full_date = data_elem.get_text(strip=True)
        if ", " in full_date:
            data_pub = full_date.split(", ", 1)[0]
        else:
            data_pub = full_date
    else:
        data_pub = "N/A"

    # Link PDF
    link_pdf = page.find("ul", class_="list-download")
    pdf_url = None
    if link_pdf:
        a_tag = link_pdf.find("a", href=True)
        if a_tag:
            pdf_url = urljoin("https://www.inail.it", a_tag["href"])

    # Testo PDF
    testo_pdf = scarica_e_leggi_pdf(pdf_url) if pdf_url else None

    return {
        "titolo": title,
        "abstract": abstract,
        "descrizione": descrizione,
        "prodotto": prodotto,
        "data_pubblicazione": data_pub,
        "pdf_url": pdf_url,
        "testo_pdf": testo_pdf
    }

In [6]:
url = 'https://www.inail.it/portale/it/inail-comunica/pubblicazioni/catalogo-generale/catalogo-generale-dettaglio.2025.09.impiego-di-gas-anestetici-fluorurati-nelle-sale-operatorie.html'
info = scrape_inail_details(driver, url)

print(info.keys())
print(info['titolo'])
print(info['abstract'])
print(info['descrizione'])
print(info['prodotto'])
print(info['data_pubblicazione'])
print(info['testo_pdf'][:500])

dict_keys(['titolo', 'abstract', 'descrizione', 'prodotto', 'data_pubblicazione', 'pdf_url', 'testo_pdf'])
Impiego di gas anestetici fluorurati nelle sale operatorie: indicazioni del regolamento (ue) 2024/573 per la sostenibilità ambientale in ottica One Health
L’obiettivo del fact sheet è quello di focalizzare l’attenzione sugli aspetti del Regolamento (UE) 2024/573 che rappresenta una parte fondamentale degli sforzi dell'Unione Europea per ridurre le emissioni di gas serra e raggiungere gli obiettivi climatici previsti dall'Accordo di Parigi e dal Green Deal europeo, per contribuire a ridurre il riscaldamento globale, abbattendo progressivamente l'uso dei gas fluorurati e promuovendo soluzioni maggiormente sostenibili per il raggiungimento della neutralità climatica entro il 2050.
In particolare l’attenzione è rivolta all’impiego di gas anestetici fluorurati utilizzati nelle sale operatorie del settore sanitario e veterinario, soprattutto riguardo al desflurano il cui impiego, per il

**Extracting publication links for a specific topic**

Once the URL for a selected topic is found, this function navigates through a specified number of pages and collects all the links to publications related to that topic.

In [7]:
from typing import List, Dict
from urllib.parse import urljoin, quote_plus
import unicodedata
from selenium.webdriver.common.by import By
from selenium.webdriver.remote.webdriver import WebDriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time

BASE_CATALOG_URL = "https://www.inail.it/portale/it/inail-comunica/pubblicazioni/catalogo-generale.html"

def normalize_query(q: str) -> str:
    q = unicodedata.normalize("NFKC", q).strip().lower()
    q = " ".join(q.split())
    q = q.replace("’", "'")
    return q

def build_inail_search_url(query: str, page: int = 1) -> str:
    if not isinstance(query, str) or not query.strip():
        raise ValueError("La query deve essere una stringa non vuota.")
    if page < 1:
        raise ValueError("Il parametro 'page' deve essere >= 1.")
    encoded = quote_plus(normalize_query(query))
    return f"{BASE_CATALOG_URL}?text={encoded}&page={page}"

def extract_cards_with_wait(driver: WebDriver, timeout: int = 8) -> List[Dict[str, str]]:
    try:
        WebDriverWait(driver, timeout).until(
            EC.any_of(
                EC.presence_of_element_located((By.CSS_SELECTOR, "h3.card-title a[href]")),
                EC.presence_of_element_located((By.CSS_SELECTOR, "div#content, main, body"))
            )
        )
    except TimeoutException:
        return []

    results = []
    for a in driver.find_elements(By.CSS_SELECTOR, "h3.card-title a[href]"):
        href = a.get_attribute("href")
        if href:
            results.append({"titolo": a.text.strip(), "url": urljoin("https://www.inail.it", href)})
    return results

def get_inail_links(driver: WebDriver, query: str, max_pages: int = 1) -> List[Dict[str, str]]:
    # Cerca e raccoglie i link delle pubblicazioni INAIL in base alla query e numero di pagine
    all_links, seen = [], set()
    for p in range(1, max_pages + 1):
        url = build_inail_search_url(query, page=p)
        driver.get(url)
        cards = extract_cards_with_wait(driver, timeout=8)
        if not cards:
            break
        for c in cards:
            if c["url"] not in seen:
                seen.add(c["url"])
                all_links.append(c)
        time.sleep(0.3)
    return all_links


# Funzione interattiva per  ricerca tramite keyword
def ask_user_and_get_inail_links(driver):
    # Chiede all’utente keyword e pagine
    query = input("Inserisci la parola o frase da cercare su INAIL: ").strip()
    while not query:
        print("La query non può essere vuota.")
        query = input("Inserisci la parola o frase da cercare su INAIL: ").strip()

    try:
        max_pages = int(input("Inserisci il numero di pagine da analizzare: ").strip())
        if max_pages < 1:
            raise ValueError
    except ValueError:
        max_pages = 1
        print("Numero non valido. Analizzerò solo 1 pagina.")

    return get_inail_links(driver, query, max_pages=max_pages)


In [14]:
links = ask_user_and_get_inail_links(driver)

# Stampa solo conteggio e lista
print(f"\nTrovate {len(links)} pubblicazioni totali.\n")
for item in links:
    print(f"{item['titolo']}\n{item['url']}\n")

Inserisci la parola o frase da cercare su INAIL: Sicurezza sul Lavoro
Inserisci il numero di pagine da analizzare: 2

Trovate 20 pubblicazioni totali.

Malprof - Le malattie psichiche sul lavoro
https://www.inail.it/portale/it/inail-comunica/pubblicazioni/catalogo-generale/catalogo-generale-dettaglio.2025.09.malprof-le-malattie-psichiche-sul-lavoro.html

Le nuove competenze e le soft skill nell’era digitale
https://www.inail.it/portale/it/inail-comunica/pubblicazioni/catalogo-generale/catalogo-generale-dettaglio.2025.09.le-nuove-competenze-e-le-soft-skill-nell-era-digitale.html

Sostanze pericolose: valori limite e valori di riferimento
https://www.inail.it/portale/it/inail-comunica/pubblicazioni/catalogo-generale/catalogo-generale-dettaglio.2025.07.factsheet-sostanze-pericolose-valori-limite-valori-riferimento.html

Stima dei potenziali lavoratori esposti ad acrilonitrile sulla base dei registri di esposizione professionale
https://www.inail.it/portale/it/inail-comunica/pubblicazioni/

**Saving scraped links to a JSON file**

Provides a function that searches INAIL publications by a user-defined keyword, paginates through a chosen number of result pages, collects article links, scrapes key fields and PDF text when available, and saves structured outputs to JSON.

In [15]:
import json
import os
import time
import random
from bs4 import BeautifulSoup
import re
from urllib.parse import urljoin

# Funzione ausiliaria per salvare in JSON
def salva_in_json(record, filename):
    if os.path.exists(filename):
        with open(filename, 'r', encoding='utf-8') as f:
            data = json.load(f)
    else:
        data = []

    data.append(record)

    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

# Funzione principale per scraping multiplo INAIL
def scrape_inail_topic(driver, max_pages=3, output_file='inail_risultati.json'):

    # Chiede all’utente le keyword, raccoglie i link e ne estrae i metadati completi.

    links = ask_user_and_get_inail_links(driver)  # Funzione interattiva
    print(f"\nTrovate {len(links)} pubblicazioni totali.\n")

    for i, item in enumerate(links, start=1):
        print(f"[{i}/{len(links)}] Estraggo: {item['titolo']}")
        try:
            dati = scrape_inail_details(driver, item['url'])
            dati['url_pubblicazione'] = item['url']
            salva_in_json(dati, output_file)
            print(f" Salvato: {item['titolo']}")
        except Exception as e:
            print(f" Errore su {item['url']}: {e}")

        time.sleep(random.uniform(2, 4))  # pausa 2–4 sec per evitare blocchi

    print(f"\nTutti i dati salvati in '{output_file}'.")


In [19]:
scrape_inail_topic(driver, max_pages=3, output_file='inail_sicurezza_lavoro.json')

Inserisci la parola o frase da cercare su INAIL: Sicurezza sul lavoro
Inserisci il numero di pagine da analizzare: 1

Trovate 10 pubblicazioni totali.

[1/10] Estraggo: Malprof - Le malattie psichiche sul lavoro
 Salvato: Malprof - Le malattie psichiche sul lavoro
[2/10] Estraggo: Le nuove competenze e le soft skill nell’era digitale
 Salvato: Le nuove competenze e le soft skill nell’era digitale
[3/10] Estraggo: Sostanze pericolose: valori limite e valori di riferimento
 Salvato: Sostanze pericolose: valori limite e valori di riferimento
[4/10] Estraggo: Stima dei potenziali lavoratori esposti ad acrilonitrile sulla base dei registri di esposizione professionale
 Salvato: Stima dei potenziali lavoratori esposti ad acrilonitrile sulla base dei registri di esposizione professionale
[5/10] Estraggo: Soluzioni basate su modelli AI: chatbot specializzato nel regolamento europeo sull’intelligenza artificiale
 Salvato: Soluzioni basate su modelli AI: chatbot specializzato nel regolamento eur

**Interactive search with date filters**

Adds an interactive layer that lets the user set both the keyword and a publication date range, builds the corresponding filtered search, iterates over results, and exports the selected publications to JSON for downstream analysis.

In [20]:
from typing import List, Dict
from urllib.parse import urljoin, quote_plus
import unicodedata
from selenium.webdriver.common.by import By
from selenium.webdriver.remote.webdriver import WebDriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time
import re

BASE_CATALOG_URL = "https://www.inail.it/portale/it/inail-comunica/pubblicazioni/catalogo-generale.html"

# Normalizzazione query testo
def normalize_query(q: str) -> str:
    q = unicodedata.normalize("NFKC", q).strip().lower()
    q = " ".join(q.split())
    q = q.replace("’", "'")
    return q

# Validazione data nel formato gg/mm/aaaa
def validate_date(date_str: str) -> bool:
    return bool(re.match(r"^\d{2}/\d{2}/\d{4}$", date_str))

# Funzione costruttore flessibile URL
def build_inail_search_url(query: str = None, page: int = 1,
                           start_date: str = None, end_date: str = None) -> str:
    if page < 1:
        raise ValueError("Il parametro 'page' deve essere >= 1.")

    params = []

    if query and query.strip():
        encoded = quote_plus(normalize_query(query))
        params.append(f"text={encoded}")

    if start_date and validate_date(start_date):
        params.append(f"startDate={quote_plus(start_date)}")
    elif start_date:
        print(f" Attenzione: '{start_date}' non è nel formato gg/mm/aaaa — ignorata.")

    if end_date and validate_date(end_date):
        params.append(f"endDate={quote_plus(end_date)}")
    elif end_date:
        print(f" Attenzione: '{end_date}' non è nel formato gg/mm/aaaa — ignorata.")

    params.append(f"page={page}")
    return f"{BASE_CATALOG_URL}?{'&'.join(params)}"

# Estrazione card
def extract_cards_with_wait(driver: WebDriver, timeout: int = 8) -> List[Dict[str, str]]:
    try:
        WebDriverWait(driver, timeout).until(
            EC.any_of(
                EC.presence_of_element_located((By.CSS_SELECTOR, "h3.card-title a[href]")),
                EC.presence_of_element_located((By.CSS_SELECTOR, "div#content, main, body"))
            )
        )
    except TimeoutException:
        return []

    results = []
    for a in driver.find_elements(By.CSS_SELECTOR, "h3.card-title a[href]"):
        href = a.get_attribute("href")
        if href:
            results.append({"titolo": a.text.strip(), "url": urljoin("https://www.inail.it", href)})
    return results

# Raccolta link da più pagine
def get_inail_links(driver: WebDriver, query: str = None, max_pages: int = 1,
                    start_date: str = None, end_date: str = None) -> List[Dict[str, str]]:
    all_links, seen = [], set()
    for p in range(1, max_pages + 1):
        url = build_inail_search_url(query, page=p, start_date=start_date, end_date=end_date)
        driver.get(url)
        cards = extract_cards_with_wait(driver, timeout=8)
        if not cards:
            break
        for c in cards:
            if c["url"] not in seen:
                seen.add(c["url"])
                all_links.append(c)
        time.sleep(0.3)
    return all_links

# Funzione completa
def ask_user_and_get_inail_links(driver):
    # Chiede all’utente keyword, periodo di pubblicazione e pagine
    query = input("Inserisci la parola o frase da cercare su INAIL (puoi lasciare vuoto): ").strip()

    start_date = input("Inserisci la data di inizio (gg/mm/aaaa) o premi Invio per saltare: ").strip()
    if start_date and not validate_date(start_date):
        print(" Formato data non valido, ignorata.")
        start_date = None

    end_date = input("Inserisci la data di fine (gg/mm/aaaa) o premi Invio per saltare: ").strip()
    if end_date and not validate_date(end_date):
        print(" Formato data non valido, ignorata.")
        end_date = None

    try:
        max_pages = int(input("Inserisci il numero di pagine da analizzare: ").strip())
        if max_pages < 1:
            raise ValueError
    except ValueError:
        max_pages = 1
        print("Numero non valido. Analizzerò solo 1 pagina.")

    return get_inail_links(driver, query, max_pages=max_pages,
                           start_date=start_date, end_date=end_date)


In [22]:
links = ask_user_and_get_inail_links(driver)

print(f"\nTrovate {len(links)} pubblicazioni totali.\n")
for item in links:
    print(f"{item['titolo']}\n{item['url']}\n")

Inserisci la parola o frase da cercare su INAIL (puoi lasciare vuoto): Salute e sicurezza
Inserisci la data di inizio (gg/mm/aaaa) o premi Invio per saltare: 10/01/2025
Inserisci la data di fine (gg/mm/aaaa) o premi Invio per saltare: 10/10/2025
Inserisci il numero di pagine da analizzare: 2

Trovate 13 pubblicazioni totali.

Bio-ritmo ospedali - Metodologia per la valutazione del rischio biologico
https://www.inail.it/portale/it/inail-comunica/pubblicazioni/catalogo-generale/catalogo-generale-dettaglio.2025.09.bio-ritmo-ospedali-monografia.html

Impiego di gas anestetici fluorurati nelle sale operatorie: indicazioni del regolamento (ue) 2024/573 per la sostenibilità ambientale in ottica One Health
https://www.inail.it/portale/it/inail-comunica/pubblicazioni/catalogo-generale/catalogo-generale-dettaglio.2025.09.impiego-di-gas-anestetici-fluorurati-nelle-sale-operatorie.html

Report azione centrale sull’attività di vigilanza. Percorso di formazione e monitoraggio sulla sicurezza dei lav

Now scrapes key fields and PDF text when available withthis new function, and saves structured outputs to JSON

In [23]:
import json
import os
import time
import random

# Funzione ausiliaria per salvare in JSON
def salva_in_json(record, filename):
    if os.path.exists(filename):
        with open(filename, 'r', encoding='utf-8') as f:
            data = json.load(f)
    else:
        data = []

    data.append(record)

    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)


# Funzione principale interattiva per scraping multiplo INAIL
def scrape_inail_topic(driver, output_file='inail_risultati.json'):
    print("=== RICERCA PUBBLICAZIONI INAIL ===")
    query = input(" Inserisci la parola o frase da cercare: ").strip()
    while not query:
        print("La query non può essere vuota.")
        query = input(" Inserisci la parola o frase da cercare: ").strip()

    try:
        max_pages = int(input(" Inserisci il numero di pagine da analizzare: ").strip())
        if max_pages < 1:
            raise ValueError
    except ValueError:
        max_pages = 1
        print("Numero non valido. Analizzerò solo 1 pagina.")

    start_date = input(" Inserisci la data di inizio (gg/mm/aaaa) o premi Invio per nessun limite: ").strip()
    end_date = input(" Inserisci la data di fine (gg/mm/aaaa) o premi Invio per nessun limite: ").strip()

    # --- Step 1: raccolta link ---
    print(" Cerco le pubblicazioni, attendi...")
    links = get_inail_links(driver, query=query, max_pages=max_pages, start_date=start_date or None, end_date=end_date or None)
    print(f" Trovate {len(links)} pubblicazioni totali per '{query}'.\n")

    # --- Step 2: scraping dettagli ---
    for i, item in enumerate(links, start=1):
        print(f"[{i}/{len(links)}] Estraggo: {item['titolo']}")
        try:
            dati = scrape_inail_details(driver, item['url'])
            dati['url_pubblicazione'] = item['url']
            salva_in_json(dati, output_file)
            print(f" Salvato: {item['titolo']}")
        except Exception as e:
            print(f" Errore su {item['url']}: {e}")

        time.sleep(random.uniform(2, 4))  # pausa random per sicurezza

    print(f" Tutti i dati salvati in '{output_file}'.")


In [28]:
scrape_inail_topic(driver)


=== RICERCA PUBBLICAZIONI INAIL ===
 Inserisci la parola o frase da cercare: Salute e Lavoro
 Inserisci il numero di pagine da analizzare: 2
 Inserisci la data di inizio (gg/mm/aaaa) o premi Invio per nessun limite: 10/01/2025
 Inserisci la data di fine (gg/mm/aaaa) o premi Invio per nessun limite: 10/10/2025
 Cerco le pubblicazioni, attendi...
 Trovate 9 pubblicazioni totali per 'Salute e Lavoro'.

[1/9] Estraggo: Report azione centrale sull’attività di vigilanza. Percorso di formazione e monitoraggio sulla sicurezza dei lavoratori in attuazione dell’art. 5 d.lgs. 81/2008
 Salvato: Report azione centrale sull’attività di vigilanza. Percorso di formazione e monitoraggio sulla sicurezza dei lavoratori in attuazione dell’art. 5 d.lgs. 81/2008
[2/9] Estraggo: Malprof - Le malattie psichiche sul lavoro
 Salvato: Malprof - Le malattie psichiche sul lavoro
[3/9] Estraggo: Le nuove competenze e le soft skill nell’era digitale
 Salvato: Le nuove competenze e le soft skill nell’era digitale
[4/

In [29]:
driver.quit()